### Features
- Load Ultralytics classification model and compute embeddings
- Compute visualization and similarity
- Apply model predictions
- Compute mistakenness

In [ ]:
import fiftyone as fo
import fiftyone.brain as fob
import fiftyone.core.models as fom
import fiftyone.types as fot
import patchbrain  # noqa: F401
from fiftyone import ViewField as F

In [ ]:
fo.annotation_config.backends["cvat"]["segment_size"] = 300

In [ ]:
dataset_type = fot.FiftyOneDataset
dataset_dir = "/home/fkwong/Downloads/marks_similarity/train_250812"
categories = [
    "danger_vehicle",
    "dumper",
    "dumper6",
    "others",
    "pickup",
    "truck_box",
    "unknown",
]
anno_key = "cvat_annotation"
prediction_field = "ultralytics_prediction"
label_field = "ground_truth"
temp_anno_field = "temp_annotation"

In [ ]:
dataset = fo.Dataset.from_dir(dataset_type=dataset_type, dataset_dir=dataset_dir)

In [ ]:
model_path = "/home/fkwong/Downloads/marks_similarity/last.pt"
config = {
    "type": "fiftyone.utils.ultralytics.FiftyOneYOLOClassificationModel",
    "config": {
        "entrypoint_fcn": "ultralytics.YOLO",
        "entrypoint_args": {"model": model_path},
        "output_processor_cls": "fiftyone.utils.ultralytics.UltralyticsClassificationOutputProcessor",
        "embeddings_layer": "<model.model.9.linear",
    },
}
model = fom.load_model(config, model_path=model_path)

In [ ]:
for k, v in model._model.named_modules():
    # print(k)
    pass
model.has_embeddings

In [ ]:
dataset.compute_embeddings(
    model,
    embeddings_field=f"{prediction_field}_embeddings",
    batch_size=32,
    num_workers=8,
    progress=True,
)

In [ ]:
dataset.delete_brain_run(f"{prediction_field}_visualization")
dataset.delete_brain_run(f"{prediction_field}_similarity")

In [ ]:
visualization_result = fob.compute_visualization(
    dataset,
    embeddings=f"{prediction_field}_embeddings",
    brain_key=f"{prediction_field}_visualization",
    progress=True,
)

In [ ]:
similarity_result = fob.compute_similarity(
    dataset,
    embeddings=f"{prediction_field}_embeddings",
    brain_key=f"{prediction_field}_similarity",
    progress=True,
)

In [ ]:
dataset.apply_model(
    model,
    label_field=prediction_field,
    batch_size=32,
    store_logits=True,
    num_workers=8,
    progress=True,
)

In [ ]:
prediction_view = dataset.match(~F(prediction_field).is_null())

In [ ]:
mistakenness_field = f"{prediction_field}_mistakenness"
fob.compute_mistakenness(
    prediction_view,
    prediction_field,
    "ground_truth",
    mistakenness_field=mistakenness_field,
)

In [ ]:
anno_view = prediction_view.match((F(mistakenness_field) > 0.5)).sort_by(
    mistakenness_field,
    reverse=False,
)

In [ ]:
sess = fo.Session(dataset, auto=False, address="0.0.0.0")

In [ ]:
anno_view = dataset.select(sess.selected)

In [ ]:
if len(anno_view) > 0:
    if anno_key in dataset.list_saved_views():
        print(f"{anno_key} view existed!")
    else:
        dataset.save_view(anno_key, anno_view)
else:
    print("No sample to be annotated.")

In [ ]:
if len(anno_view) > 0:
    sess = fo.Session(anno_view, auto=False, address="0.0.0.0")
else:
    print("No sample to be reviewed.")

In [ ]:
if len(anno_view) > 0:
    anno_results = anno_view.annotate(
        anno_key=anno_key,
        label_field=label_field,
        label_type="classification",
        classes=categories,
        launch_editor=False,
    )
else:
    print("No sample to be annotated.")

In [ ]:
anno_view = dataset.load_annotation_view(anno_key)
anno_view.load_annotations(
    anno_key,
    dest_field=temp_anno_field,
    cleanup=False,
)
sess.view = anno_view

In [ ]:
anno_results = anno_view.load_annotation_results(anno_key)
for k, v in anno_results.get_status()[label_field].items():
    if v["status"] != "completed":
        print(f"Task-{k} is not completed yet, current status: {v['status']}")
        break
else:
    anno_view.load_annotations(anno_key, dest_field=label_field, cleanup=False)

In [ ]:
anno_results = dataset.load_annotation_results(
    anno_key,
    cache=False,
)

In [ ]:
dataset.delete_sample_fields(
    [
        temp_anno_field,
        mistakenness_field,
        prediction_field,
    ],
    error_level=1,
)
anno_results.cleanup()
if anno_key in dataset.list_annotation_runs():
    dataset.delete_annotation_run(anno_key)
if anno_key in dataset.list_saved_views():
    dataset.delete_saved_view(anno_key)
dataset.list_saved_views()
dataset.list_annotation_runs()

In [ ]:
dataset.count_sample_tags()

In [ ]:
dataset.export(
    export_dir=dataset_dir + "_new",
    dataset_type=fot.FiftyOneDataset,
    export_media=True,
)